<i>Copyright (c) Microsoft Corporation.</i>

<i>Licensed under the MIT License.</i>

# Data Preparation for Retail Sales Forecasting

This notebook introduces how to split the Orange Juice dataset into training sets and test sets for training and evaluating different retail sales forecasting methods.

We use backtesting a method that tests a predictive model on historical data to evaluate the forecasting methods. Other than standard [K-fold cross validation](https://en.wikipedia.org/wiki/Cross-validation_%28statistics%29) which randomly splits data into K folds, we split the data so that any of the time stamps in the training set is no later than any of the time stamps in the test set to ensure that no future information is used (expect certain information that we can know beforehand, e.g., price of the product in the next few weeks as we can set the price manually).



## Global Settings and Imports

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import sys

from fclib.common.utils import git_repo_path
from fclib.dataset.ojdata import download_ojdata, split_train_test

print("System version: {}".format(sys.version))

System version: 3.6.10 |Anaconda, Inc.| (default, Jan  7 2020, 21:14:29) 
[GCC 7.3.0]


In [3]:
# Use False if you've already downloaded and split the data
DOWNLOAD_DATA = True

# Data directory
DATA_DIR = os.path.join(git_repo_path(), "ojdata")

## Download Data

We need to download the Orange Juice data before splitting it into training and test sets. By default, the following cell will download the data. If you've already done so, you may skip this part by switching `DOWNLOAD_DATA` to `False`.

The dataset is from R package [bayesm](https://cran.r-project.org/web/packages/bayesm/index.html) and is part of the [Dominick's dataset](https://www.chicagobooth.edu/research/kilts/datasets/dominicks). It contains the following two csv files:

1. `yx.csv` includes weekly sales of refrigerated orange juice at 83 stores. This files has 106139 rows and 19 columns. It contains weekly sales and prices of 11 orange juice brands as well as information about profit, deal, and advertisement for each brand. Note that the weekly sales is captured by a column named `logmove` which corresponds to the natural logarithm of the number of units sold. To get the number of units sold, you need to apply an exponential transform to this column.

2. `storedemo.csv` includes demographic information on those stores. This table has 83 rows and 13 columns. For every store, the table describes demographic information of its consumers, distance to the nearest warehouse store, average distance to the nearest 5 supermarkets, ratio of its sales to the nearest warehouse store, and ratio of its sales
to the average of the nearest 5 stores.

Note that the week number starts from 40 in this dataset, while the full Dominick's dataset has week number from 1 to 400. According to [Dominick's Data Manual](https://www.chicagobooth.edu/-/media/enterprise/centers/kilts/datasets/dominicks-dataset/dominicks-manual-and-codebook_kiltscenter.aspx), week 1 starts on 09/14/1989.
Please see pages 40 and 41 of the [bayesm reference manual](https://cran.r-project.org/web/packages/bayesm/bayesm.pdf) and the [Dominick's Data Manual](https://www.chicagobooth.edu/-/media/enterprise/centers/kilts/datasets/dominicks-dataset/dominicks-manual-and-codebook_kiltscenter.aspx) for more details about the data.

In [4]:
if DOWNLOAD_DATA:
    download_ojdata(DATA_DIR)
    print("Data download completed. Data saved to " + DATA_DIR)

Data already exists at the specified location.
Data download completed. Data saved to /data/home/vapaunic/forecasting/ojdata


## Parameters

Here we define parameters for data preparation and forecast settings. In particular:
- *N_SPLITS* defines the number of training/testing splits we want to split our data into
- *HORIZON* or forecasting horizon determines the number of weeks to forecast in the future, or the test period length
- *GAP* defines the gap (in weeks) between the training and test data. This is to allow business managers to plan for the forecasted demand.
- *FIRST_WEEK* is the first available week in the data
- *LAST_WEEK* is the last available week in the data

In [5]:
# Forecasting settings
N_SPLITS = 1
HORIZON = 2
GAP = 2
FIRST_WEEK = 40
LAST_WEEK = 156

## Splitting Time Series Data

In all our examples we use `split_train_test()` utility function to split the Orange Juice dataset. Splitting time series data into training and testing has to follow the rule that, in each split, test indices need to be higher than the train indices, and higher than the test indices in the previous split.

We wrote the `split_train_test()` function to do just that. Given the parameters listed above, it creates `N_SPLITS` number of training/testing data splits, so that each test split is `HORIZON` weeks long, and the testing period is `GAP` number of weeks away from the training period. The first available week in the data (or the first week we want to start modeling from) is given as `FIRST_WEEK`, and the last week we want to work with is `LAST_WEEK`.

For demonstration, this is what the time series split on the Orange Juice dataset looks like, for the parameters listed above.
For `HORIZON = 2` and `GAP = 2`, assuming the current week is week `153`, our goal is to forecast the sales in week `155` and `156` using the training data. As you can see, the first forecasting week is `two` weeks away from the current week, as we want to leave time for planning inventory in practice.

![Single split](../../../../assets/time_series_split_singleround.jpg)

We also refer to splits as rounds, so for `N_SPLITS = 1`, we have single-round forecasting, and for `N_SPLITS > 1`, we have multi-round forecasting.

## Single-Round Forecasting

Next, we can use `split_train_test()` utility function to split the data in `yx.csv` into training and test sets based on the parameters described above. If we want to do a one-time model training and evaluation, we can split the data using the default settings provided in the function.

The data split function will return training data and test data as dataframes. The training data includes `train_df` and `aux_df` with `train_df` containing the historical sales and `aux_df` containing price/promotion information. Here we assume that future price and promotion information up to a certain number of weeks ahead is predetermined and known. The test data is stored in `test_df` which contains the sales of each product.


In [6]:
train_df_list, test_df_list, aux_df_list = split_train_test(
    data_dir=DATA_DIR,
    n_splits=N_SPLITS,
    horizon=HORIZON,
    gap=GAP,
    first_week=FIRST_WEEK,
    last_week=LAST_WEEK,
    write_csv=True,
)

# Split returns a list, extract the dataframes from the list
train_df = train_df_list[0].reset_index()
test_df = test_df_list[0].reset_index()
aux_df = aux_df_list[0].reset_index()

In [7]:
train_df.head(10)

,store,brand,week,logmove,constant,price1,price2,price3,price4,price5,price6,price7,price8,price9,price10,price11,deal,feat,profit
0,2,1,40,9.018695,1,0.060469,0.060497,0.042031,0.029531,0.049531,0.053021,0.038906,0.041406,0.028906,0.024844,0.038984,1,0.0,37.992326
1,2,1,46,8.723231,1,0.060469,0.060312,0.045156,0.046719,0.049531,0.047813,0.045781,0.027969,0.042969,0.042031,0.038984,0,0.0,30.126667
2,2,1,47,8.253228,1,0.060469,0.060312,0.045156,0.046719,0.037344,0.053021,0.045781,0.041406,0.048125,0.032656,0.038984,0,0.0,30.000000
3,2,1,48,8.987197,1,0.060469,0.060312,0.049844,0.037344,0.049531,0.053021,0.045781,0.041406,0.042344,0.032656,0.038984,0,0.0,29.950000
4,2,1,50,9.093357,1,0.060469,0.060312,0.043594,0.031094,0.049531,0.053021,0.046648,0.041406,0.042344,0.032656,0.038203,0,0.0,29.920000
5,2,1,51,8.877382,1,0.060469,0.060312,0.043594,0.046719,0.049531,0.053021,0.046456,0.035781,0.042344,0.029531,0.038203,0,0.0,29.920000
6,2,1,52,9.294682,1,0.051406,0.060312,0.043594,0.046719,0.049531,0.053021,0.047969,0.035781,0.031094,0.029531,0.038984,1,0.0,27.125471
7,2,1,53,8.954674,1,0.051406,0.060312,0.049844,0.046719,0.034219,0.053021,0.047969,0.035781,0.031094,0.029531,0.038984,1,0.0,27.125041
8,2,1,54,9.049232,1,0.051406,0.060312,0.049844,0.037344,0.049531,0.053021,0.047969,0.038281,0.048125,0.027969,0.035078,1,0.0,27.082481
9,2,1,57,8.613230,1,0.051406,0.060312,0.048281,0.046719,0.031094,0.053021,0.031094,0.041406,0.042344,0.042031,0.038984,1,0.0,27.061163


In [8]:
test_df.head(10)

,store,brand,week,logmove,constant,price1,price2,price3,price4,price5,price6,price7,price8,price9,price10,price11,deal,feat,profit
0,2,1,155,9.003070,1,0.042901,0.051979,0.045156,0.031094,0.026406,0.050352,0.041366,0.031094,0.037344,0.018594,0.029453,0,0.00000,26.851024
1,2,1,156,9.875911,1,0.038906,0.051979,0.046719,0.031819,0.028320,0.046771,0.040469,0.031094,0.037344,0.032656,0.028047,1,1.00000,22.896316
2,2,2,155,8.769041,1,0.042901,0.051979,0.045156,0.031094,0.026406,0.050352,0.041366,0.031094,0.037344,0.018594,0.029453,0,0.00000,29.861343
3,2,2,156,8.723231,1,0.038906,0.051979,0.046719,0.031819,0.028320,0.046771,0.040469,0.031094,0.037344,0.032656,0.028047,0,0.00000,29.916250
4,2,3,155,8.090709,1,0.042901,0.051979,0.045156,0.031094,0.026406,0.050352,0.041366,0.031094,0.037344,0.018594,0.029453,1,1.00000,36.118103
5,2,3,156,7.336937,1,0.038906,0.051979,0.046719,0.031819,0.028320,0.046771,0.040469,0.031094,0.037344,0.032656,0.028047,0,0.00000,39.047500
6,2,4,155,9.869310,1,0.042901,0.051979,0.045156,0.031094,0.026406,0.050352,0.041366,0.031094,0.037344,0.018594,0.029453,1,1.00000,23.502053
7,2,4,156,9.135617,1,0.038906,0.051979,0.046719,0.031819,0.028320,0.046771,0.040469,0.031094,0.037344,0.032656,0.028047,1,0.97379,26.420000
8,2,5,155,9.852615,1,0.042901,0.051979,0.045156,0.031094,0.026406,0.050352,0.041366,0.031094,0.037344,0.018594,0.029453,1,0.00000,19.999910
9,2,5,156,9.357380,1,0.038906,0.051979,0.046719,0.031819,0.028320,0.046771,0.040469,0.031094,0.037344,0.032656,0.028047,1,0.00000,24.212895


In [9]:
aux_df.head(10)

,store,brand,week,price1,price2,price3,price4,price5,price6,price7,price8,price9,price10,price11,deal,feat
0,2,1,40,0.060469,0.060497,0.042031,0.029531,0.049531,0.053021,0.038906,0.041406,0.028906,0.024844,0.038984,1,0.0
1,2,1,46,0.060469,0.060312,0.045156,0.046719,0.049531,0.047813,0.045781,0.027969,0.042969,0.042031,0.038984,0,0.0
2,2,1,47,0.060469,0.060312,0.045156,0.046719,0.037344,0.053021,0.045781,0.041406,0.048125,0.032656,0.038984,0,0.0
3,2,1,48,0.060469,0.060312,0.049844,0.037344,0.049531,0.053021,0.045781,0.041406,0.042344,0.032656,0.038984,0,0.0
4,2,1,50,0.060469,0.060312,0.043594,0.031094,0.049531,0.053021,0.046648,0.041406,0.042344,0.032656,0.038203,0,0.0
5,2,1,51,0.060469,0.060312,0.043594,0.046719,0.049531,0.053021,0.046456,0.035781,0.042344,0.029531,0.038203,0,0.0
6,2,1,52,0.051406,0.060312,0.043594,0.046719,0.049531,0.053021,0.047969,0.035781,0.031094,0.029531,0.038984,1,0.0
7,2,1,53,0.051406,0.060312,0.049844,0.046719,0.034219,0.053021,0.047969,0.035781,0.031094,0.029531,0.038984,1,0.0
8,2,1,54,0.051406,0.060312,0.049844,0.037344,0.049531,0.053021,0.047969,0.038281,0.048125,0.027969,0.035078,1,0.0
9,2,1,57,0.051406,0.060312,0.048281,0.046719,0.031094,0.053021,0.031094,0.041406,0.042344,0.042031,0.038984,1,0.0


## Multi-Round Forecasting

To create training data and test data for multi-round forecasting, we use the same function passing a number greater than `1` to `n_splits` parameter. Note that the forecasting period we generate in each test round are **non-overlapping**. This allows us to evaluate the forecasting model on multiple rounds of data, and get a more robust estimate of our model's performance.

For demonstration, this is what the time series splits would look like for `N_SPLITS = 5`, and using other settings as above:

![Multi split](../../../../assets/time_series_split_multiround.jpg)


Let's now generate `10` rounds of training/testing data.

In [10]:
## Generate 10 splits (rounds) of data
N_SPLITS = 10

train_df_list, test_df_list, aux_df_list = split_train_test(
    DATA_DIR, n_splits=N_SPLITS, horizon=HORIZON, gap=GAP, first_week=FIRST_WEEK, last_week=LAST_WEEK, write_csv=True,
)

The above cell wil generate the following splits.

| **Round** | **Train period in weeks** | **Test period in weeks** |
| -------- | -------------------------- | ----------------------- |
| 1 | 40 - 135 | 137 - 138 |
| 2 | 40 - 137 | 139 - 140 |
| 3 | 40 - 139 | 141 - 142 |
| 4 | 40 - 141 | 143 - 144 |
| 5 | 40 - 143 | 145 - 146 |
| 6 | 40 - 145 | 147 - 148 |
| 7 | 40 - 147 | 149 - 150 |
| 8 | 40 - 149 | 151 - 152 |
| 9 | 40 - 151 | 153 - 154 |
| 10 | 40 - 153 | 155 - 156 |

The gap of one week between training period and test period allows store managers to prepare the stock based on the forecasted demand. Besides, we assume that the information about the price, deal, and advertisement up until the forecast period end week is available at each round.

In [11]:
for i in range(len(train_df_list)):
    train_df = train_df_list[i]
    test_df = test_df_list[i]
    aux_df = aux_df_list[i]
    print(f"ROUND {i+1}")
    print("--------")
    print("Training")
    print(f"    Data shape: {train_df.shape}")
    print(f"    Week range: {min(train_df['week'])}-{max(train_df['week'])}")
    print("Testing")
    print(f"    Data shape: {test_df.shape}")
    print(f"    Week range: {min(test_df['week'])}-{max(test_df['week'])}")
    print("Auxiliary")
    print(f"    Data shape: {aux_df.shape}")
    print(f"    Week range: {min(aux_df['week'])}-{max(aux_df['week'])}\n")

ROUND 1
--------
Training
    Data shape: (84183, 18)
    Week range: 40-135
Testing
    Data shape: (1826, 18)
    Week range: 137-138
Auxiliary
    Data shape: (86911, 15)
    Week range: 40-138

ROUND 2
--------
Training
    Data shape: (85998, 18)
    Week range: 40-137
Testing
    Data shape: (1793, 18)
    Week range: 139-140
Auxiliary
    Data shape: (88704, 15)
    Week range: 40-140

ROUND 3
--------
Training
    Data shape: (87802, 18)
    Week range: 40-139
Testing
    Data shape: (1771, 18)
    Week range: 141-142
Auxiliary
    Data shape: (90475, 15)
    Week range: 40-142

ROUND 4
--------
Training
    Data shape: (89617, 18)
    Week range: 40-141
Testing
    Data shape: (1749, 18)
    Week range: 143-144
Auxiliary
    Data shape: (92224, 15)
    Week range: 40-144

ROUND 5
--------
Training
    Data shape: (91333, 18)
    Week range: 40-143
Testing
    Data shape: (1727, 18)
    Week range: 145-146
Auxiliary
    Data shape: (93951, 15)
    Week range: 40-146

ROUND 6
--

## Additional Reading

\[1\] Christoph GBergmeir, Rob J. Hyndman, and Bonsoo Koo. 2018. A Note on the Validity of Cross-Validation for Evaluating Autoregressive Time Series Prediction. Computational Statistics & Data Analysis. 120, pp. 70-83.<br>
\[2\] How To Backtest Machine Learning Models for Time Series Forecasting: https://machinelearningmastery.com/backtest-machine-learning-models-time-series-forecasting/Parameters.rst <br>

